# Module 4 - Hands on Machine Learning

In [1]:
import sklearn

## Input Formats

## Features 

## Regression

Predict Temperature

## Classification

In a state. 

## Fitting Models to Data

## Graphs and Scikitlean